<a href="https://colab.research.google.com/github/prasad1482/Text-summarization/blob/main/Text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing hugging face transformer

In [4]:
!pip install transformers


#Import Libraries

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch



#Load predefined models

In [6]:
# Load BART model and tokenizer from Hugging Face
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

#Input long input

In [12]:
# Take input from user
print("🔹 Enter the text you'd like to summarize (minimum 3 lines):\n")
input_text = input()

# Optional: Limit long inputs
if len(input_text.split()) > 1024:
    print("\n⚠️ Input is too long. Truncating to 1024 tokens for BART compatibility.\n")


🔹 Enter the text you'd like to summarize (minimum 3 lines):

A summary is a concise overview of the main points of a text, capturing its essence without including unnecessary details. It's a shortened version of the original text, typically written in your own words, and serves to inform the reader about the core message and key ideas.  Here's a more detailed explanation: Key Characteristics of a Summary: Conciseness: Summaries are shorter than the original text, focusing on the most important information.  Objectivity: A summary should present the main points without adding personal opinions or interpretations.  Clarity: The language used in a summary should be clear and easy to understand, making the main points accessible to the reader.  Accuracy: A summary should accurately reflect the original text's meaning and key arguments.  Original Language:


#Preprocess and tokenize input

In [13]:
# Tokenize and encode the input text
inputs = tokenizer.batch_encode_plus(
    [input_text],
    max_length=1024,
    return_tensors='pt',
    truncation=True
)


#Generate the summary

In [14]:
# Generate summary using BART model
summary_ids = model.generate(
    inputs['input_ids'],
    num_beams=4,
    length_penalty=2.0,
    max_length=150,
    min_length=40,
    no_repeat_ngram_size=3,
    early_stopping=True
)


#Decode and print summary

In [15]:
# Decode summary tokens to readable text
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("🔹 Summary Output:\n")
print(summary)


🔹 Summary Output:

A summary is a shortened version of the original text, typically written in your own words. It serves to inform the reader about the core message and key ideas. A summary should present the main points without adding personal opinions or interpretations.


Till now my basic model is completed now i want to add simple UI to it

In [16]:
!pip install gradio --quiet


In [17]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load model & tokenizer once
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Summarizer function
def summarize_text(input_text):
    inputs = tokenizer([input_text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        length_penalty=2.0,
        max_length=150,
        min_length=40,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [18]:
import gradio as gr

interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=10, placeholder="Paste your long text here..."),
    outputs=gr.Textbox(label="Summary"),
    title="Text Summarizer with BART",
    description="Enter any long article or paragraph, and this AI will summarize it using facebook/bart-large-cnn."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c5255c9c93ecdbd62.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
